In [1]:
import torch_geometric.nn as gnn
import torch_geometric
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.linalg
from torch_geometric.nn import GATConv
import numpy as np
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
device

device(type='cuda')

In [2]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [3]:
# from torch_geometric.datasets import Planetoid
# name_data = 'Cora'
# dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Wisconsin'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import Planetoid
# name_data = 'PubMed'
# dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)

from torch_geometric.datasets import Actor
dataset = Actor(root = '/tmp' + "Actor")

# from torch_geometric.datasets import WebKB
# name_data = 'Cornell'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WebKB
# name_data = 'Texas'
# dataset = WebKB(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WikipediaNetwork
# name_data = 'Chameleon'
# dataset = WikipediaNetwork(root= '/tmp/' + name_data, name = name_data)

# from torch_geometric.datasets import WikipediaNetwork
# name_data = 'Squirrel'
# dataset = WikipediaNetwork(root= '/tmp/' + name_data, name = name_data)

Processing...
Done!


In [4]:
data = dataset[0]

edges = data.edge_index

#dimension: num_nodes * num_features
features = data.x

num_nodes = features.shape[0]
num_features = features.shape[1]
#dimension: num_nodes * num_nodes
adj = torch.zeros((num_nodes,num_nodes))
degree = torch.zeros((num_nodes,num_nodes))
for i in tqdm(range(edges.shape[1])):
    first = edges[0][i]
    second = edges[1][i]
    adj[first][second] = 1
    adj[second][first] = 1
for i in tqdm(range(num_nodes)):
    degree[i][i] = sum(adj[i][:])

100%|██████████| 7600/7600 [03:20<00:00, 37.81it/s]


In [5]:
def data_sample(num_nodes):
  sample_num = int(num_nodes*0.6)
  sample_lst = np.random.choice(num_nodes,sample_num, replace = False)
  train_mask = [False] * num_nodes
  test_mask = [True] * num_nodes
  for i in sample_lst:
    train_mask[i] = True
    test_mask[i] = False
  train_mask = torch.tensor(train_mask)
  test_mask = torch.tensor(test_mask)
  return train_mask, test_mask


In [6]:
def reset_parameter(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()


In [7]:
degree = degree.to(device)
adj = adj.to(device)
data = data.to(device)

In [8]:
class FBLayer(nn.Module):
    def __init__(self, dim_in, dim_out):
      super().__init__()
      self.low = nn.Linear(dim_in, dim_out, bias = False)
      gain = nn.init.calculate_gain("relu")
      nn.init.xavier_normal_(self.low.weight, gain)
      self.activation = nn.ReLU()
        
    def forward(self, input, adj, degree, d_inv):
      lap = degree - adj
      Llp = torch.mm(torch.mm(d_inv, lap), d_inv)
      Hl = torch.mm(Llp, self.activation(self.low(input)))
      return Hl

    def reset_parameter():
      gain = nn.init.calculate_gain("relu")
      nn.init.xavier_normal_(self.low.weight, gain)


In [9]:
class GAT(torch.nn.Module):
    def __init__(self, num_head, nhid):
        super().__init__()
        self.gc1 = GATConv(dataset.num_node_features, nhid, heads=num_head)
        self.gc2 = GATConv(nhid*num_head, dataset.num_classes)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gc1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, training=self.training)
        x = self.gc2(x, edge_index)
        return F.log_softmax(x, dim=1)
    

In [10]:
class FBGAT(torch.nn.Module):
    def __init__(self, num_head, nhid):
      super().__init__()
      self.fb1 = FBLayer(dataset.num_node_features, nhid*num_head)
      self.fb2 = FBLayer(nhid*num_head ,dataset.num_classes)
      self.conv1 = GATConv(dataset.num_node_features, nhid, heads = num_head)
      self.conv2 = GATConv(nhid*num_head, dataset.num_classes)
      self.aL_1 = torch.nn.Parameter(torch.rand(1))
      self.aH_1 = torch.nn.Parameter(torch.rand(1))
      self.aL_2 = torch.nn.Parameter(torch.rand(1))
      self.aH_2 = torch.nn.Parameter(torch.rand(1))

    def forward(self, data, adj, degree, d_inv):
      input, edge_index = data.x, data.edge_index
      x_1 = self.fb1(input, adj, degree, d_inv) #high pass
      y_1 = self.conv1(input, edge_index) #low pass layer
      z_1 = self.aL_1 * x_1 + self.aH_1 * y_1
      z_1 = F.elu(z_1)
      z_1 = F.dropout(z_1, training=self.training)
      x_2 = self.fb2(z_1, adj, degree, d_inv)
      y_2 = self.conv2(z_1, edge_index)
      z_2 = self.aL_2 * x_2 + self.aH_2 * y_2

      return F.log_softmax(z_2, dim=1)

In [11]:
acc_list_2 = []
for i in range(5):
    model_2 = GAT(48, 8).to(device)
    optimizer_2 = torch.optim.Adam(model_2.parameters(), lr=0.05, weight_decay=5e-5)
    model_2.train()
    print("Number of Trial: " + str(i))
    reset_parameter(model_2)
    train_mask, test_mask = data_sample(num_nodes)
    for epoch in tqdm(range(100)):
        optimizer_2.zero_grad()
        out = model_2(data)
        loss = F.nll_loss(out[train_mask], data.y[train_mask]) 
        loss.backward()
        optimizer_2.step()
    model_2.eval()
    _, pred = model_2(data).max(dim=1)
    correct = int(pred[test_mask].eq(data.y[test_mask]).sum().item())
    acc = correct / int(test_mask.sum())
    acc_list_2.append(acc)
    del model_2,correct

100%|██████████| 100/100 [00:03<00:00, 26.88it/s]


In [12]:
print(np.mean(acc_list_2))
del acc_list_2

0.25861842105263155


In [13]:
acc_list = []
inter = scipy.linalg.fractional_matrix_power(degree.cpu(), (-1/2))
d_inv = torch.from_numpy(inter).to(device)
for i in range(5):
    model = FBGAT(48, 8).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=5e-5)
    model.train()
    print("Number of Trial: " + str(i))
    reset_parameter(model)
    train_mask, test_mask = data_sample(num_nodes)
    for epoch in tqdm(range(100)):
        optimizer.zero_grad()
        out = model(data, adj, degree, d_inv)
        loss = F.nll_loss(out[train_mask], data.y[train_mask]) 
        loss.backward()
        optimizer.step()
    model.eval()
    _, pred = model(data, adj, degree, d_inv).max(dim=1)
    correct = int(pred[test_mask].eq(data.y[test_mask]).sum() .item())
    acc = correct / int(test_mask.sum())
    acc_list.append(acc)
    del model, correct
del d_inv

100%|██████████| 100/100 [03:09<00:00,  1.90s/it]


In [14]:
print('FBGAT Accuracy: {:.4f}'.format(np.mean(acc_list)))

FBGAT Accuracy: 0.3020


In [15]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

NameError: name 'model' is not defined